In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This example assumes that you have an Ollama model running (with `ollama serve`) on some host that is reachable (whether a `localhost` of your computer or some other host with public IP).

In [32]:
from llamba.chatmodel_wrappers.ollama import OllamaModel
import llamba.plots as bplt
import config

In [33]:
chatbot = OllamaModel(url="http://127.0.0.1:11434/api/generate", model="llama3")
connection = chatbot.check_connection()
print(connection)

True


### Get a bioage model

In [34]:
from pathlib import Path
import os
import shap
import numpy as np
import pickle
import torch
from txai_omics_3.src.models.tabular.widedeep.ft_transformer import WDFTTransformerModel
from llamba.bioage_model import BioAgeModel

root_dir = Path(os.getcwd()).parent

# Model
fn_model = f"{root_dir}/samples/txai_omics_3/data/immuno/model.ckpt"
model = WDFTTransformerModel.load_from_checkpoint(checkpoint_path=fn_model)
bioage_model = BioAgeModel(model=model)

# SHAP
fn_shap = f"{root_dir}/samples/txai_omics_3/data/immuno/shap.pickle"

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()
with open(fn_shap, 'rb') as handle:
    shap_dict = pickle.load(handle)

Lightning automatically upgraded your loaded checkpoint from v1.6.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint h:\Lobachevsky\llamba\samples\txai_omics_3\data\immuno\model.ckpt`
Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.


### Load data

In [35]:
import pandas as pd

my_data = {'CXCL9': 2599.629474, 
           'CCL22': 820.306524, 
           'IL6': 0.846377, 
           'PDGFB': 13400.666359, 
           'CD40LG': 1853.847406, 
           'IL27': 1128.886982,
           'VEGFA': 153.574220,
           'CSF1': 239.627236,
           'PDGFA': 1005.844290,
           'CXCL10': 228.229829,
           'Age': 90.454972 }

my_df = pd.DataFrame(my_data, index=[0])

# Model data

train_data = pd.read_excel(f"{root_dir}/samples/txai_omics_3/data/immuno/data.xlsx")

### Initialize a connector

In [36]:
from llamba.connector import LlambaConnector

connector = LlambaConnector(bioage_model=bioage_model, chat_model=chatbot)

### Run an analysis task

In [37]:
res = connector.analyze(data=my_df, shap_dict=shap_dict)
print(res['analysis'])

Your bioage is 79 and your aging acceleration is -11, which means you are ageing slower than normal.

Here is some more information about your data. 

CXCL9: 2599.629474
CXCL9, also known as monokine induced by gamma interferon (MIG), is a cytokine that belongs to the CXC chemokine family. It is primarily produced by monocytes and macrophages in response to inflammatory stimuli.

An increased level of CXCL9 in older adults may indicate an ongoing inflammatory process, potentially driven by age-related diseases such as atherosclerosis, osteoporosis, or chronic infections. Elevated levels of CXCL9 can also be associated with increased risk for age-related disorders like cancer, cardiovascular disease, and Alzheimer's disease.

In the context of aging, CXCL9 might play a role in promoting immune cell infiltration into tissues, potentially contributing to the development of frailty and age-related diseases.

CD40LG: 1853.847406
CD40LG, also known as TNFSF5 or CD154, is a ligand for the CD4